In [1]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 599.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 KB 579.8 kB/s eta 0:00:00a 0:00:01


In [ ]:
import geopandas as gpd
import h3
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
from datetime import datetime

class GridManager:
    def __init__(self, base_resolution=9, min_resolution=7, max_resolution=11):
        self.base_resolution = base_resolution
        self.min_resolution = min_resolution
        self.max_resolution = max_resolution
        self.grid_data = {}
        self.traffic_history = {}
        
    def load_city_boundary(self, boundary_path):
        """Load and process city boundary"""
        self.boundary = gpd.read_file(boundary_path)
        self.boundary = self.boundary.to_crs(epsg=4326)
        
    def create_base_grid(self):
        """Create initial H3 hexagonal grid"""
        # Get boundary polygon coordinates
        boundary_coords = self.boundary.geometry.iloc[0]
        
        # Generate H3 hexagons
        hexagons = list(h3.polyfill(
            boundary_coords.__geo_interface__,
            self.base_resolution,
            geo_json_conformant=True
        ))
        
        # Create grid data structure
        self.grid_data = {
            hex_id: {
                'resolution': self.base_resolution,
                'traffic_level': 0,
                'center': h3.h3_to_geo(hex_id),
                'boundaries': h3.h3_to_geo_boundary(hex_id)
            } for hex_id in hexagons
        }
        
        return self.grid_data
    
    def update_traffic_levels(self, traffic_data):
        """Update traffic levels for each grid cell"""
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        for hex_id in self.grid_data:
            # Simulate traffic data for demonstration
            # In real implementation, use actual traffic data
            traffic_level = traffic_data.get(hex_id, 0)
            
            self.grid_data[hex_id]['traffic_level'] = traffic_level
            
            # Store historical data
            if hex_id not in self.traffic_history:
                self.traffic_history[hex_id] = []
            self.traffic_history[hex_id].append((timestamp, traffic_level))
    
    def adjust_grid_resolution(self):
        """Dynamically adjust grid resolution based on traffic levels"""
        for hex_id in list(self.grid_data.keys()):
            current_traffic = self.grid_data[hex_id]['traffic_level']
            current_res = self.grid_data[hex_id]['resolution']
            
            # Refine grid if traffic is high
            if current_traffic > 100 and current_res < self.max_resolution:
                self._refine_grid(hex_id)
            
            # Merge grids if traffic is low
            elif current_traffic < 20 and current_res > self.min_resolution:
                self._merge_grid(hex_id)
    
    def _refine_grid(self, hex_id):
        """Split a hexagon into smaller hexagons"""
        if self.grid_data[hex_id]['resolution'] >= self.max_resolution:
            return
            
        children = h3.h3_to_children(hex_id)
        parent_traffic = self.grid_data[hex_id]['traffic_level']
        
        # Remove parent
        del self.grid_data[hex_id]
        
        # Add children
        for child in children:
            self.grid_data[child] = {
                'resolution': self.grid_data[hex_id]['resolution'] + 1,
                'traffic_level': parent_traffic / len(children),  # Distribute traffic
                'center': h3.h3_to_geo(child),
                'boundaries': h3.h3_to_geo_boundary(child)
            }
    
    def _merge_grid(self, hex_id):
        """Merge hexagons into larger hexagon"""
        if self.grid_data[hex_id]['resolution'] <= self.min_resolution:
            return
            
        parent = h3.h3_to_parent(hex_id)
        siblings = h3.h3_to_children(parent)
        
        # Calculate total traffic for parent
        total_traffic = sum(
            self.grid_data[h]['traffic_level']
            for h in siblings
            if h in self.grid_data
        )
        
        # Remove all siblings
        for sibling in siblings:
            if sibling in self.grid_data:
                del self.grid_data[sibling]
        
        # Add parent
        self.grid_data[parent] = {
            'resolution': self.grid_data[hex_id]['resolution'] - 1,
            'traffic_level': total_traffic,
            'center': h3.h3_to_geo(parent),
            'boundaries': h3.h3_to_geo_boundary(parent)
        }
    
    def visualize_grid(self, save_path=None):
        """Create interactive map visualization"""
        # Create base map centered on city
        center = self.boundary.geometry.iloc[0].centroid
        m = folium.Map(
            location=[center.y, center.x],
            zoom_start=11
        )
        
        # Add hexagons to map
        for hex_id, data in self.grid_data.items():
            # Convert traffic level to color
            color = self._get_color_for_traffic(data['traffic_level'])
            
            # Create polygon
            folium.Polygon(
                locations=[[lat, lng] for lng, lat in data['boundaries']],
                color=color,
                fill=True,
                fill_color=color,
                fill_opacity=0.4,
                popup=f"Hex ID: {hex_id}<br>Traffic: {data['traffic_level']}<br>Resolution: {data['resolution']}"
            ).add_to(m)
        
        if save_path:
            m.save(save_path)
        return m
    
    def _get_color_for_traffic(self, traffic_level):
        """Convert traffic level to color"""
        if traffic_level > 80:
            return 'red'
        elif traffic_level > 50:
            return 'orange'
        elif traffic_level > 20:
            return 'yellow'
        return 'green'

# Example usage
if __name__ == "__main__":
    # Initialize grid manager
    grid_mgr = GridManager()
    
    # Load city boundary
    grid_mgr.load_city_boundary('/home/raw/Desktop/Coding/Jhakaas_Rasta/geopkg/clipping_boundary.geojson')
    
    # Create initial grid
    grid_mgr.create_base_grid()
    
    # Simulate some traffic data
    mock_traffic = {
        hex_id: np.random.randint(0, 100)
        for hex_id in grid_mgr.grid_data.keys()
    }
    
    # Update traffic levels
    grid_mgr.update_traffic_levels(mock_traffic)
    
    # Adjust grid resolution
    grid_mgr.adjust_grid_resolution()
    
    # Visualize results
    grid_mgr.visualize_grid('traffic_grid.html')